In [1]:
from acquire import get_iris_data
from prepare import prep_iris
from acquire import get_titanic_data
from prepare import prep_titanic_data

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

df = get_titanic_data()

In [2]:
df = prep_titanic_data(df)
df.dropna(inplace=True)

In [3]:
train, test = train_test_split(df)

In [4]:
scaler = MinMaxScaler()

scaler.fit(train[['age', 'fare']])

train[['age', 'fare']] = scaler.transform(train[['age', 'fare']])
test[['age', 'fare']] = scaler.transform(test[['age', 'fare']])

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyda

In [5]:
logit = LogisticRegression()

logit.fit(train[['pclass', 'age', 'fare', 'sibsp', 'parch']], train.survived)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [6]:
logit.predict_proba(train[['pclass', 'age', 'fare', 'sibsp', 'parch']])

array([[0.59642877, 0.40357123],
       [0.71738064, 0.28261936],
       [0.41425111, 0.58574889],
       ...,
       [0.81490461, 0.18509539],
       [0.78524908, 0.21475092],
       [0.46548868, 0.53451132]])

In [7]:
train['prediction'] = logit.predict(train[['pclass', 'age', 'fare', 'sibsp', 'parch']])

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
logit.score(train[['pclass', 'age', 'fare', 'sibsp', 'parch']], train.survived)

0.7065420560747664

In [9]:
print(classification_report(train.survived, train.prediction))

confusion_matrix(train.survived, train.prediction)

              precision    recall  f1-score   support

           0       0.71      0.84      0.77       310
           1       0.70      0.52      0.60       225

   micro avg       0.71      0.71      0.71       535
   macro avg       0.71      0.68      0.68       535
weighted avg       0.71      0.71      0.70       535



array([[261,  49],
       [108, 117]])

In [16]:
titanic_matrix = confusion_matrix(train.survived, train.prediction)
print(f'True Negative: {titanic_matrix[0][0]}')
print(f'False Positive: {titanic_matrix[0][1]}')
print(f'False Negative: {titanic_matrix[1][0]}')
print(f'True Positive: {titanic_matrix[1][1]}')
true_pos = titanic_matrix[1][1]
true_neg = titanic_matrix[0][0]
false_pos = titanic_matrix[0][1]
false_neg = titanic_matrix[1][0]

True Negative: 261
False Positive: 49
False Negative: 108
True Positive: 117


In [26]:
total_obs = titanic_matrix[0][0] + titanic_matrix[0][1] + titanic_matrix[1][0] + titanic_matrix[1][1]
accuracy = (true_pos + true_neg) / total_obs
print(f'Accuracy: {accuracy}')
precision = true_pos / (true_pos + false_pos)
print(f'Precision: {precision}')
recall = true_pos / (true_pos + false_neg)
print(f'Recall (true positive rate): {recall}')
f1 = recall + precision / 2
print(f'f1: {f1}')
fp_rate = false_pos / (false_pos + true_neg)
print(f'False Positive Rate (specificity): {fp_rate}')
tn_rate = true_neg / (true_neg + false_pos)
print(f'True Negative Rate: {tn_rate}')
fn_rate = false_neg / (true_pos + false_neg)
print('False Negative Rate: {:2}'.format(fn_rate))
print(f'support: {total_obs}')


Accuracy: 0.7065420560747664
Precision: 0.7048192771084337
Recall (true positive rate): 0.52
f1: 0.8724096385542168
False Positive Rate (specificity): 0.15806451612903225
True Negative Rate: 0.8419354838709677
False Negative Rate: 0.48
support: 535


In [37]:
import matplotlib.pyplot as plt
# y_pred_proba = [i for i in train['prediction']]
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.scatter(y_pred_proba, train['prediction'])

## Let's run through the model building process with another 'solver' parameter.  Our best option should be the liblinear option (which is the default) as we are working with a small data set and not a multiclass problem

In [38]:
X = df[['pclass','age','fare','sibsp','parch']]
y = df[['survived']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state = 123)

X_train.head()

,pclass,age,fare,sibsp,parch
60,3,22.0,7.2292,0,0
348,3,3.0,15.9000,1,1
606,3,30.0,7.8958,0,0
195,1,58.0,146.5208,0,0
56,2,21.0,10.5000,0,0


In [39]:
logit = LogisticRegression(C=1, class_weight={1:2}, random_state = 123, solver='saga')
logit.fit(X_train, y_train)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1, class_weight={1: 2}, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=123, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

In [40]:
y_pred = logit.predict(X_train)
y_pred_proba = logit.predict_proba(X_train)

In [41]:
logit.score(X_train, y_train)

0.6472945891783567

In [ ]:
# saga performed at 0.64 accuracy, 
#less than our first model on default liblinear

In [42]:
dfi = get_iris_data()

In [45]:
dfi = prep_iris(dfi)

In [46]:
dfi.head()

,species,sepal_length,sepal_width,petal_length,petal_width,species_encode
0,setosa,5.1,3.5,1.4,0.2,0
1,setosa,4.9,3.0,1.4,0.2,0
2,setosa,4.7,3.2,1.3,0.2,0
3,setosa,4.6,3.1,1.5,0.2,0
4,setosa,5.0,3.6,1.4,0.2,0


In [57]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

X = dfi.drop(columns=['species', 'species_encode'])
y = dfi[['species']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 123)
X_train.head()

,sepal_length,sepal_width,petal_length,petal_width
114,5.8,2.8,5.1,2.4
136,6.3,3.4,5.6,2.4
53,5.5,2.3,4.0,1.3
19,5.1,3.8,1.5,0.3
38,4.4,3.0,1.3,0.2


In [58]:
clf = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=123)


In [59]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')

In [60]:
y_pred = clf.predict(X_train)
y_pred[0:4]

array(['virginica', 'virginica', 'versicolor', 'setosa'], dtype=object)

In [64]:
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba

array([[0.   , 0.   , 1.   ],
       [0.   , 0.   , 1.   ],
       [0.   , 0.975, 0.025],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [0.   , 0.   , 1.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [0.   , 0.975, 0.025],
       [0.   , 0.975, 0.025],
       [0.   , 0.975, 0.025],
       [0.   , 0.975, 0.025],
       [0.   , 0.   , 1.   ],
       [0.   , 0.975, 0.025],
       [0.   , 0.   , 1.   ],
       [1.   , 0.   , 0.   ],
       [0.   , 0.   , 1.   ],
       [0.   , 0.975, 0.025],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [0.   , 0.   , 1.   ],
       [0.   , 0.975, 0.025],
       [0.   , 0.   , 1.   ],
       [0.   , 0.   , 1.   ],
       [1.   , 0.   , 0.   ],
       [0.   , 0.975, 0.025],
       [0.   , 0.975, 0.025],
       [0.   , 0.   , 1.   ],
       [1.   , 0.   , 0.   ],
       [0.   , 0.   , 1.   ],
       [0.   , 0.975, 0.025],
       [0.   , 0.975, 0.025],
       [1.   , 0.   , 0.   ],
       [0.

In [65]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.98


In [66]:
confusion_matrix(y_train, y_pred)

array([[32,  0,  0],
       [ 0, 40,  0],
       [ 0,  2, 31]])

In [67]:
sorted(y_train.species.unique())

['setosa', 'versicolor', 'virginica']

In [68]:
y_train.species.value_counts()

versicolor    40
virginica     33
setosa        32
Name: species, dtype: int64

In [69]:
import pandas as pd

labels = sorted(y_train.species.unique())

pd.DataFrame(confusion_matrix(y_train, y_pred), index=labels, columns=labels)

,setosa,versicolor,virginica
setosa,32,0,0
versicolor,0,40,0
virginica,0,2,31


In [70]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        32
  versicolor       0.95      1.00      0.98        40
   virginica       1.00      0.94      0.97        33

   micro avg       0.98      0.98      0.98       105
   macro avg       0.98      0.98      0.98       105
weighted avg       0.98      0.98      0.98       105



In [74]:
import graphviz

from graphviz import Graph

dot_data = tree.export_graphviz(clf, out_file=None)
graph = graphviz.Source(dot_data)
graph.render('iris_decision_tree', view=True)

'iris_decision_tree.pdf'

In [105]:
import numpy as np
import pandas as pd
tree_cnf_matrix = confusion_matrix(y_train, y_pred)
print(tree_cnf_matrix)
print(np.diag(tree_cnf_matrix))
tree_cnf_matrix.sum(axis=1)

[[32  0  0]
 [ 0 40  0]
 [ 0  2 31]]
[32 40 31]


array([32, 40, 33])

In [109]:

FP = tree_cnf_matrix.sum(axis=0) - np.diag(tree_cnf_matrix)  
FN = (tree_cnf_matrix.sum(axis=1)) - np.diag(tree_cnf_matrix)
TP = np.diag(tree_cnf_matrix)
TN = tree_cnf_matrix.sum() - (FP + FN + TP)

FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
print(f'Recall: {TPR}')
# Specificity or true negative rate
TNR = TN/(TN+FP) 
print(f'True Negative Rate: {TNR}')
# Precision or positive predictive value
PPV = TP/(TP+FP)
print(f'Precision: {PPV}')
# Negative predictive value
NPV = TN/(TN+FN)
print(f'Negative Predictive Value: {NPV}')
# Fall out or false positive rate
FPR = FP/(FP+TN)
print(f'False positive Rate: {FPR}')
# False negative rate
FNR = FN/(TP+FN)
print(f'False Negative Rate: {FNR}')
# False discovery rate
FDR = FP/(TP+FP)
print(f'False Discovery Rate: {FDR}')
# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)
print(f'Overall Accuracy: {ACC}')

Recall: [1.         1.         0.93939394]
True Negative Rate: [1.         0.96923077 1.        ]
Precision: [1.         0.95238095 1.        ]
Negative Predictive Value: [1.         1.         0.97297297]
False positive Rate: [0.         0.03076923 0.        ]
False Negative Rate: [0.         0.         0.06060606]
False Discovery Rate: [0.         0.04761905 0.        ]
Overall Accuracy: [1.         0.98095238 0.98095238]


In [86]:
metrics.precision_recall_fscore_support(y_train, y_pred)

(array([1.        , 0.95238095, 1.        ]),
 array([1.        , 1.        , 0.93939394]),
 array([1.        , 0.97560976, 0.96875   ]),
 array([32, 40, 33]))